In [1]:
from urllib.request import urlopen
import pandas as pd
from bs4 import BeautifulSoup
import webbrowser
from html_table_parser import parser_functions as parser
import numpy as np

#### 등록된 api로 접속

In [2]:
API_KEY='b0e7fa75*****************************' #authority key
company_code='005930' #company code
start_date='20170101'

#### 사업 보고서 종류 확인

In [3]:
url ='https://opendart.fss.or.kr/api/list.xml?crtfc_key='+API_KEY+'&corp_code='+company_code+'&bgn_de='+start_date+'&pblntf_detail_ty=A001&pblntf_detail_ty=A002&pblntf_detail_ty=A003'
resultXML=urlopen(url)
result=resultXML.read()

In [4]:
xmlsoup=BeautifulSoup(result,'html.parser')

In [5]:
data = pd.DataFrame()
for t in xmlsoup.findAll('list'):
    temp=pd.DataFrame(([[t.corp_code.string,t.corp_name.string,t.stock_code.string,t.report_nm.string,
         t.rcept_no.string,t.flr_nm.string]]),
         columns=["corp_code","corp_name","stock_code","report_nm","rcept_no","flr_nm"])
    data=pd.concat([data,temp])

data=data.reset_index(drop=True)

In [6]:
data.head()

,corp_code,corp_name,stock_code,report_nm,rcept_no,flr_nm
0,00126380,삼성전자,005930,분기보고서 (2019.09),20191114001273,삼성전자
1,00126380,삼성전자,005930,반기보고서 (2019.06),20190814002218,삼성전자
2,00126380,삼성전자,005930,분기보고서 (2019.03),20190515001605,삼성전자
3,00126380,삼성전자,005930,사업보고서 (2018.12),20190401004781,삼성전자
4,00126380,삼성전자,005930,분기보고서 (2018.09),20181114001530,삼성전자


#### 각각의 url을 활용하여 재무제표 뽑기

In [7]:
a2019_09="http://dart.fss.or.kr/report/viewer.do?rcpNo=20191114001273&dcmNo=6958001&eleId=15&offset=1099327&length=69959&dtd=dart3.xsd"
a2019_06='http://dart.fss.or.kr/report/viewer.do?rcpNo=20190814002218&dcmNo=6846651&eleId=15&offset=1080041&length=69283&dtd=dart3.xsd'
a2019_03='http://dart.fss.or.kr/report/viewer.do?rcpNo=20190515001605&dcmNo=6738798&eleId=15&offset=1026852&length=68820&dtd=dart3.xsd'
a2018_12='http://dart.fss.or.kr/report/viewer.do?rcpNo=20190401004781&dcmNo=6616741&eleId=15&offset=1332659&length=90711&dtd=dart3.xsd'
a2018_09='http://dart.fss.or.kr/report/viewer.do?rcpNo=20181114001530&dcmNo=6382016&eleId=15&offset=1108496&length=72539&dtd=dart3.xsd'
a2018_06='http://dart.fss.or.kr/report/viewer.do?rcpNo=20180814001113&dcmNo=6282263&eleId=15&offset=1102087&length=72265&dtd=dart3.xsd'
a2018_03='http://dart.fss.or.kr/report/viewer.do?rcpNo=20180515001699&dcmNo=6180129&eleId=15&offset=1047985&length=72075&dtd=dart3.xsd'
a2017_12='http://dart.fss.or.kr/report/viewer.do?rcpNo=20180402005019&dcmNo=6060273&eleId=15&offset=1350131&length=91652&dtd=dart3.xsd'
a2017_09='http://dart.fss.or.kr/report/viewer.do?rcpNo=20171114001694&dcmNo=5845320&eleId=15&offset=1135961&length=73638&dtd=dart3.xsd'
a2017_06='http://dart.fss.or.kr/report/viewer.do?rcpNo=20170814001832&dcmNo=5752868&eleId=15&offset=1111624&length=73272&dtd=dart3.xsd'
a2017_03='http://dart.fss.or.kr/report/viewer.do?rcpNo=20170515003806&dcmNo=5653406&eleId=15&offset=1101560&length=112166&dtd=dart3.xsd'

In [8]:
url_list=[a2017_03, a2017_06, a2017_09, a2017_12, a2018_03, a2018_06, a2018_09, a2018_12, a2019_03, a2019_06, a2019_09]

#### 재무상태표 칼럼

In [9]:
a2019_09_col=['구분','제 51 기 3분기말','제 50 기말']
a2019_06_col=['구분','제 51 기 반기말','제 50 기말']
a2019_03_col=['구분','제 51 기 1분기말','제 50 기말']
a2018_12_col=['구분','제 50 기','제 49 기','제 48 기']
a2018_09_col=['구분','제 50 기 3분기말','제 49 기말']
a2018_06_col=['구분','제 50 기 반기말','제 49 기말']
a2018_03_col=['구분','제 50 기 1분기말','제 49 기말']
a2017_12_col=['구분','제 49 기','제 48 기','제 47 기']
a2017_09_col=['구분','제 49 기 3분기말','제 48 기말']
a2017_06_col=['구분','제 49 기 반기말','제 48 기말']
a2017_03_col=['구분','제 49 기 1분기말','제 48 기말','제 47 기말']

columns_a=[a2017_03_col,a2017_06_col,a2017_09_col,a2017_12_col,a2018_03_col,a2018_06_col,a2018_09_col,a2018_12_col
         ,a2019_03_col,a2019_06_col,a2019_09_col]

#### 손익계산서 칼럼

In [10]:
b2017_03_col=['구분','제 49 기 1분기3개월','제 49 기 1분기누적','제 48 기 1분기3개월','제 48 기 1분기누적','제 48 기','제 47 기']
b2017_06_col=['구분','제 49 기 반기3개월','제 49 기 반기누적','제 48 기 반기3개월','제 48 기 반기누적']
b2017_09_col=['구분','제 49 기 3분기3개월', '제 49 기 3분기누적', '제 48 기 3분기3개월', '제 48 기 3분기누적']
b2017_12_col=['구분','제 49 기','제 48 기','제 47 기']
b2018_03_col=['구분','제 50 기 1분기3개월', '제 50 기 1분기누적', '제 49 기 1분기3개월', '제 49 기 1분기누적']
b2018_06_col=['구분','제 50 기 반기3개월', '제 50 기 반기누적', '제 49 기 반기3개월', '제 49 기 반기누적']
b2018_09_col=['구분','제 50 기 3분기3개월','제 50 기 3분기누적','제 49 기 3분기3개월','제 49 기 3분기누적']
b2018_12_col=['구분','제 50 기','제 49 기','제 48 기']
b2019_03_col=['구분','제 51 기 1분기3개월', '제 51 기 1분기누적', '제 50 기 1분기3개월', '제 50 기 1분기누적']
b2019_06_col=['구분','제 51 기 반기3개월', '제 51 기 반기누적' ,'제 50 기 반기3개월', '제 50 기 반기누적']
b2019_09_col=['구분','제 51 기 3분기3개월', '제 51 기 3분기누적', '제 50 기 3분기3개월', '제 50 기 3분기누적']

columns_b=[b2017_03_col,b2017_06_col,b2017_09_col,b2017_12_col,b2018_03_col,b2018_06_col,b2018_09_col,b2018_12_col
         ,b2019_03_col,b2019_06_col,b2019_09_col]

#### 현금흐름표 칼럼

In [11]:
c2019_09_col=['구분','제 51 기 3분기','제 50 기 3분기']
c2019_06_col=['구분','제 51 기 반기','제 50 기 반기']
c2019_03_col=['구분','제 51 기 1분기','제 50 기 1분기']
c2018_12_col=['구분','제 50 기','제 49 기','제 48 기']
c2018_09_col=['구분','제 50 기 3분기','제 49 기 3분기']
c2018_06_col=['구분','제 50 기 반기','제 49 기 반기']
c2018_03_col=['구분','제 50 기 1분기','제 49 기 1분기']
c2017_12_col=['구분','제 49 기','제 48 기','제 47 기']
c2017_09_col=['구분','제 49 기 3분기','제 48 기 3분기']
c2017_06_col=['구분','제 49 기 반기','제 48 기 반기']
c2017_03_col=['구분','제 49 기 1분기','제 48 기 1분기','제 48 기','제 47 기']

columns_c=[c2017_03_col,c2017_06_col,c2017_09_col,c2017_12_col,c2018_03_col,c2018_06_col,c2018_09_col,c2018_12_col
         ,c2019_03_col,c2019_06_col,c2019_09_col]

## dataframe 정리

#### 재무상태표

In [12]:
data1 = pd.DataFrame()

for i,j in zip(url_list, columns_a):
    
    report=urlopen(i)
    r=report.read()
    xmlsoup=BeautifulSoup(r,'html.parser')
    body=xmlsoup.find("body")
    table=body.find_all("table")
    
    # 재무상태표
    p1 = parser.make2d(table[1],) 
    BS = pd.DataFrame(p1[1:],columns=j)
    data1=pd.concat([data1,BS],axis=1)

In [13]:
BS_final=data1[['구분','제 49 기 1분기말', '제 49 기 반기말', '제 49 기 3분기말', '제 49 기말', '제 50 기 1분기말',
             '제 50 기 반기말', '제 50 기 3분기말', '제 50 기말', '제 51 기 1분기말', '제 51 기 반기말','제 51 기 3분기말']]

BS_final = BS_final.loc[:,~BS_final.columns.duplicated()]

---------------------

#### 손익계산서

In [14]:
# 크롤링을 할때 재무상태표는 순서가 일정하지만 손익계산서와 현금흐름표는 일정하지 않아서 수동으로 하나하나 찾아줘야함.

In [15]:
data2=pd.DataFrame()

for a in [0,1,2,4,8,9,10]:
    report=urlopen(url_list[a])
    r=report.read()
    xmlsoup=BeautifulSoup(r,'html.parser')
    body=xmlsoup.find("body")
    table=body.find_all("table")


    # 손익계산서
    p2 = parser.make2d(table[3]) 
    IS = pd.DataFrame(p2[2:])
    IS.columns=columns_b[a]
    data2=pd.concat([data2,IS],axis=1)

In [16]:
data3=pd.DataFrame()

for a in [5,6]:
    report=urlopen(url_list[a])
    r=report.read()
    xmlsoup=BeautifulSoup(r,'html.parser')
    body=xmlsoup.find("body")
    table=body.find_all("table")


    # 손익계산서
    p2 = parser.make2d(table[4]) 
    IS = pd.DataFrame(p2[2:])
    IS.columns=columns_b[a]
    data3=pd.concat([data3,IS],axis=1)

In [17]:
report=urlopen(url_list[3])
r=report.read()
xmlsoup=BeautifulSoup(r,'html.parser')
body=xmlsoup.find("body")
table=body.find_all("table")


# 손익계산서
p2_3 = parser.make2d(table[3]) 
IS_3 = pd.DataFrame(p2_3[1:])
IS_3.columns=columns_b[3]

In [18]:
report=urlopen(url_list[7])
r=report.read()
xmlsoup=BeautifulSoup(r,'html.parser')
body=xmlsoup.find("body")
table=body.find_all("table")


# 손익계산서
p2_7 = parser.make2d(table[4]) 
IS_7 = pd.DataFrame(p2_7[1:])
IS_7.columns=columns_b[7]

In [19]:
IS_final=pd.concat([data2,data3,IS_3,IS_7],axis=1)

IS_final = IS_final.loc[:,~IS_final.columns.duplicated()]

In [20]:
IS_final_1=IS_final[['구분','제 49 기 1분기3개월','제 49 기 반기3개월','제 49 기 3분기3개월','제 49 기',
                     '제 50 기 1분기3개월','제 50 기 반기3개월','제 50 기 3분기3개월',
                     '제 51 기 1분기3개월','제 51 기 반기3개월','제 51 기 3분기3개월']]
IS_final_2=IS_final[['구분','제 49 기 1분기누적','제 49 기 반기누적','제 49 기 3분기누적', '제 49 기',
                     '제 50 기 1분기누적','제 50 기 반기누적','제 50 기 3분기누적','제 50 기',
                     '제 51 기 1분기누적','제 51 기 반기누적','제 51 기 3분기누적']]

------------------

#### 현금흐름표

In [21]:
data4=pd.DataFrame()

for a in [0,1,2,3,4]:
    report=urlopen(url_list[a])
    r=report.read()
    xmlsoup=BeautifulSoup(r,'html.parser')
    body=xmlsoup.find("body")
    table=body.find_all("table")


    # 현금흐름표
    p3 = parser.make2d(table[-3])
    CF1 = pd.DataFrame(p3[1:])
    CF1.columns=columns_c[a]
    data4=pd.concat([data4,CF1], axis=1)

In [22]:
data5=pd.DataFrame()

for b in [5,6,8,9,10]:
    report=urlopen(url_list[b])
    r=report.read()
    xmlsoup=BeautifulSoup(r,'html.parser')
    body=xmlsoup.find("body")
    table=body.find_all("table")


    # 현금흐름표
    p3 = parser.make2d(table[-4])
    CF2 = pd.DataFrame(p3[1:])
    CF2.columns=columns_c[b]
    data5=pd.concat([data5,CF2], axis=1)

In [23]:
report=urlopen(url_list[7])
r=report.read()
xmlsoup=BeautifulSoup(r,'html.parser')
body=xmlsoup.find("body")
table=body.find_all("table")


# 현금흐름표
p3_7 = parser.make2d(table[-5])
CF_7 = pd.DataFrame(p3_7[1:])
CF_7.columns=columns_c[7]

In [24]:
CF_final=pd.concat([data4,data5,CF_7],axis=1)

CF_final=CF_final[['구분','제 49 기 1분기',  '제 49 기 반기', '제 49 기 3분기','제 49 기','제 50 기 1분기', 
                   '제 50 기 반기', '제 50 기 3분기', '제 50 기','제 51 기 1분기',  '제 51 기 반기',  '제 51 기 3분기']]

CF_final = CF_final.loc[:,~CF_final.columns.duplicated()]

In [25]:
BS_final.head()

,구분,제 49 기 1분기말,제 49 기 반기말,제 49 기 3분기말,제 49 기말,제 50 기 1분기말,제 50 기 반기말,제 50 기 3분기말,제 50 기말,제 51 기 1분기말,제 51 기 반기말,제 51 기 3분기말
0,자산,,,,,,,,,,,
1,유동자산,"60,869,192","59,385,664","65,782,329","70,155,189","72,370,085","68,070,224","81,904,347","80,039,455","82,311,689","72,045,329","76,848,738"
2,현금및현금성자산,"4,874,602","2,617,278","2,955,954","2,763,768","7,769,590","3,981,948","3,920,032","2,607,957","5,573,010","3,609,930","2,135,881"
3,단기금융상품,"21,570,384","16,570,360","20,110,259","25,510,064","20,136,857","20,137,337","29,461,618","34,113,871","28,107,265","19,802,670","24,403,587"
4,단기매도가능금융자산,,,,"27,881,777","30,045,336","28,863,536","33,069,425","24,933,267","28,119,553","28,444,363","31,554,688"


In [26]:
IS_final_1.head()

,구분,제 49 기 1분기3개월,제 49 기 반기3개월,제 49 기 3분기3개월,제 49 기,제 50 기 1분기3개월,제 50 기 반기3개월,제 50 기 3분기3개월,제 51 기 1분기3개월,제 51 기 반기3개월,제 51 기 3분기3개월
0,수익(매출액),"34,578,524","42,130,809","43,074,291","161,915,007","42,606,978","41,314,723","47,645,339","37,038,395","38,149,674","42,204,911"
1,매출원가,"22,941,536","27,741,036","26,678,477","101,399,657","25,196,260","24,606,173","27,578,197","26,452,223","28,690,721","32,735,989"
2,매출총이익,"11,636,988","14,389,773","16,395,814","60,515,350","17,410,718","16,708,550","20,067,142","10,586,172","9,458,953","9,468,922"
3,판매비와관리비,"5,961,417","6,462,217","6,401,333","25,658,259","6,209,821","5,658,945","6,154,425","6,689,249","6,661,795","6,802,955"
4,영업이익(손실),"5,675,571","7,927,556","9,994,481","34,857,091","11,200,897","11,049,605","13,912,717","3,896,923","2,797,158","2,665,967"


In [27]:
IS_final_2.head()

,구분,제 49 기 1분기누적,제 49 기 반기누적,제 49 기 3분기누적,제 49 기,제 50 기 1분기누적,제 50 기 반기누적,제 50 기 3분기누적,제 50 기,제 51 기 1분기누적,제 51 기 반기누적,제 51 기 3분기누적
0,수익(매출액),"34,578,524","76,709,333","119,783,624","161,915,007","42,606,978","83,921,701","131,567,040","170,381,870","37,038,395","75,188,069","117,392,980"
1,매출원가,"22,941,536","50,682,572","77,361,049","101,399,657","25,196,260","49,802,433","77,380,630","101,666,506","26,452,223","55,142,944","87,878,933"
2,매출총이익,"11,636,988","26,026,761","42,422,575","60,515,350","17,410,718","34,119,268","54,186,410","68,715,364","10,586,172","20,045,125","29,514,047"
3,판매비와관리비,"5,961,417","12,423,634","18,824,967","25,658,259","6,209,821","11,868,766","18,023,191","25,015,913","6,689,249","13,351,044","20,153,999"
4,영업이익(손실),"5,675,571","13,603,127","23,597,608","34,857,091","11,200,897","22,250,502","36,163,219","43,699,451","3,896,923","6,694,081","9,360,048"


In [28]:
CF_final.head()

,구분,제 49 기 1분기,제 49 기 반기,제 49 기 3분기,제 49 기,제 50 기 1분기,제 50 기 반기,제 50 기 3분기,제 50 기,제 51 기 1분기,제 51 기 반기,제 51 기 3분기
0,영업활동 현금흐름,"7,087,039","11,002,113","23,660,843","38,906,190","13,056,587","18,073,882","29,342,891","44,341,217","3,106,972","2,228,736","12,895,344"
1,영업에서 창출된 현금흐름,"8,014,616","12,939,220","25,941,877","41,350,471","12,448,884","24,025,910","37,007,574","53,596,311","2,976,890","8,527,608","17,444,767"
2,당기순이익,"4,873,767","11,017,491","18,794,866","28,800,837","8,452,458","16,530,443","26,741,991","32,815,127","3,088,628","7,151,797","11,737,687"
3,조정,"3,696,372","8,390,233","13,656,686","18,012,976","6,447,338","13,416,116","21,355,836","27,095,149","4,891,329","7,933,853","10,225,453"
4,영업활동으로 인한 자산부채의 변동,"(555,523)","(6,468,504)","(6,509,675)","(5,463,342)","(2,450,912)","(5,920,649)","(11,090,253)","(6,313,965)","(5,003,067)","(6,558,042)","(4,518,373)"
